## **Chapter 11 - Agentic AI**
This notebook explores Agentic AI through structured examples, showcasing how AI agents plan, retrieve data, and execute tasks autonomously. Using `LangChain` and `CrewAI`, the notebook demonstrates key abstractions, including datasets, prompts, model selection, agent-based reasoning, and task execution. Examples range from structured data retrieval using Hugging Face datasets to interactive AI agents that process information and act independently. By the end, you'll see how these frameworks power AI-driven decision-making and automation, making AI applications more adaptive and autonomous.

**Setting Up API Keys** for Hugging Face, Google, and OpenAI
Before running the code examples in this chapter, API keys must be configured for Hugging Face Hub, Google APIs (Serper and Gemini), and OpenAI. This script retrieves stored credentials in Google Colab Secrets and sets them as environment variables for seamless integration with AI services.

***Note:*** See Google Colab Secrets for instructions on how to store and manage API keys securely.

In [ ]:
# API Key Setup for Hugging Face Hub, Google API, and OpenAI in Google Colab
# Constants and API Key Configuration
import os
from google.colab import userdata

# === Load API keys securely from Google Colab Secrets ===
def load_api_keys():
    keys = {
        "HF_TOKEN": userdata.get("HF_TOKEN"),
        "OPENAI_API_KEY": userdata.get("OPENAI_API_KEY"),
        "SERPAPI_API_KEY": userdata.get("SERPAPI_API_KEY"),
        "SERPER_API_KEY": userdata.get("SERPER_API_KEY")
    }
    for key, value in keys.items():
        if not value:
            raise ValueError(f"❌ Missing {key}. Please set this API key in Colab secrets.")
        os.environ[key] = value
    print("✅ All API keys loaded and configured successfully.")

# Execute API key loading upon running this cell
load_api_keys()


### Listing 9-1: Loading and Analyzing Game Data
This code retrieves and processes the **Steam Games** dataset from Hugging Face. It selects the top five highest-rated games, sorting them by positive reviews. The dataset provides structured information on game titles, genres, and player feedback, forming the foundation for AI-driven analysis and decision-making.

***Note:*** Before running this code, ensure you have the necessary dependencies installed by running:

In [ ]:
%pip install --quiet datasets

In [ ]:
from datasets import load_dataset
import pandas as pd

# Load the Steam Games dataset from Hugging Face
dataset = load_dataset("FronkonGames/steam-games-dataset",
                       split="train")

# Convert to Pandas DataFrame
df = dataset.to_pandas()

# Select relevant columns
df = df[["Name", "Genres", "Positive"]].sort_values(
     by="Positive", ascending=False).head(5)

# Rename columns for clarity
df.columns = ["Game", "Genres", "Positive_Reviews"]

# Display structured dataset output
print("\nTop 5 Highest Rated Games on Steam:")
print(df.to_string(index=False))

### Listing 11-2: Building Prompt Templates and Querying AI Models with LangChain

This listing walks through a progression of prompt-engineering techniques using
LangChain and Hugging Face models. It begins by installing the required packages
and setting up a Hugging Face text-generation endpoint. The first cells introduce
basic prompt templates, then extend them with one-shot and few-shot examples.
Next, the code demonstrates variable substitution to create flexible, category-
driven trivia prompts. The listing then integrates structured data from the Steam
Games Dataset, showing how real-world information can be fed into AI-driven game
concept generation. Finally, the listing runs all templates end-to-end, producing
structured responses from several prompt formats.

***Note:*** Install the required dependencies before running the listing. The pip installation step may produce a few warnings or version-conflict messages in Colab, but in our experience the code cells still run correctly once the packages are in place.

In [ ]:
# Install required packages for Hugging Face and LangChain usage

print("Installing packages... this can take a minute or two.")

%pip install -q langchain langchain-community langchain-huggingface langchain-openai google-search-results

print("All required packaged installed and ready!")

In [ ]:
# === Hugging Face trivia chatbot with LangChain ===

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

# Models that work with the Hugging Face Inference API for text generation
DEFAULT_MODELS = [
    "mistralai/Mistral-7B-Instruct-v0.2",
    "meta-llama/Meta-Llama-3-8B-Instruct",
    "openai/gpt-oss-128k",
]

DEFAULT_MODEL = DEFAULT_MODELS[0]          # Pick one model from the list
TEMP = 0.2                                 # Lower = more focused answers

base_llm = HuggingFaceEndpoint(
    repo_id=DEFAULT_MODEL,                 # Hugging Face model id
    task="text-generation",                # Use text-generation endpoint
    temperature=TEMP,                      # Sampling temperature
    max_new_tokens=128,                    # Max tokens in each reply
    return_full_text=False,                # Only return new text
    # huggingfacehub_api_token="YOUR_TOKEN",  # Or set env var instead
)

chat_llm = ChatHuggingFace(llm=base_llm)   # Wrap endpoint as a chat model
parser = StrOutputParser()                 # Parse output into a plain string

In [ ]:
# === Basic Trivia Prompt Example ===

basic_prompt = ChatPromptTemplate.from_messages([
    (   "system",
        "You are a trivia expert who provides informative answers.",
    ),
    (   "human",
        "Who invented the lightbulb?",
    ),
    (   "ai",
        "The lightbulb was invented by Thomas Edison in 1879.",
    ),
    (   "human",
        "{question}",
    ),
])

basic_chain = basic_prompt | chat_llm | parser  # Prompt → LLM → text

response = basic_chain.invoke(
    {
        "question": "What is the tallest mountain on Earth?",
    }
)
print("📘 Trivia Response:\n", response)

In [ ]:
# === Few-shot trivia example with Hugging Face ===

few_shot_prompt = ChatPromptTemplate.from_messages([
    # System Prompt
    ('system', "You are a trivia expert providing clear answers."),
    # First Example
    ('human', "Who discovered gravity?"),
    ('ai', "Gravity was discovered by Sir Isaac Newton in the 17th century."),
    # Second Example
    ('human', "What is the speed of light?"),
    ('ai', "The speed of light is approximately 299,792 kilometers per second."),
    # Third Example
    ('human', "Who developed the theory of relativity?"),
    ('ai', "Albert Einstein developed the theory of relativity."),
    # The Question posed to AI model
    ('human', "Who are the authors of ‘Build Your Own AI?’.")
])

few_shot_chain = few_shot_prompt | chat_llm | parser

response = few_shot_chain.invoke({})
print("📗 Few-shot Response:\n", response)

In [ ]:
# === Variable trivia prompt with Hugging Face ===

variable_prompt = ChatPromptTemplate.from_messages([
    (   "system",
        "You are a trivia expert providing players with challenging questions.",
    ),
    (   "human",
        "Category: {trivia_category}",
    ),
    (   "human",
        "Question: {trivia_question}, answer in 50 words or less.",
    ),
])

variable_chain = variable_prompt | chat_llm | parser

response = variable_chain.invoke(
    {
        "trivia_category": "Astronomy",
        "trivia_question": "What causes a solar eclipse?",
    }
)

print("🎯 Trivia Response:\n", response)

In [ ]:
# === Game analysis with Hugging Face ===

from datasets import load_dataset
import pandas as pd

# Load the Steam dataset
dataset = load_dataset(
    "FronkonGames/steam-games-dataset", split="train"
)
df = dataset.to_pandas()

# Extract the top five highest-rated games
df = df[["Name", "Genres", "Positive"]].sort_values(
    by="Positive", ascending=False
).head(5)

# Format the games into a compact list for the prompt
game_list = "\n".join(
    f"{row['Name']} (Genre: {row['Genres']}, Reviews: {row['Positive']})"
    for _, row in df.iterrows()
)

# Create the game-analysis prompt
game_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "Using the top games below, design a simple three-agent game.",
    ),
    (
        "human",
        "Top-rated games:\n{game_list}\n\n"
        "Provide:\n"
        "- A recommended genre\n"
        "- A game title\n"
        "- A basic mechanic\n"
        "- Three agent roles (2 players, 1 GM)\n"
        "- A short play explanation",
    ),
])

# Build the chain (prompt → LLM → text)
game_chain = game_prompt | chat_llm | parser

# Invoke with the formatted game list
response = game_chain.invoke({"game_list": game_list})

print("🎮 Game Concept:\n", response)

In [ ]:
# === Running all trivia prompt templates ===

# Basic trivia prompt
basic_chain = basic_prompt | chat_llm | parser
basic_out = basic_chain.invoke(
    {"question": "Who invented the telephone?"}
)

# Few-shot trivia prompt
few_shot_chain = few_shot_prompt | chat_llm | parser
few_shot_out = few_shot_chain.invoke({})

# Variable trivia prompt
variable_chain = variable_prompt | chat_llm | parser
variable_out = variable_chain.invoke(
    {
        "trivia_category": "Physics",
        "trivia_question": "What is quantum tunneling?",
    }
)

print("Basic:", basic_out)
print("Few-shot:", few_shot_out)
print("Variable:", variable_out)

### Listing 11-3: Implementing AI Agents and Automating a Trivia Contest
This code sets up Neural Duel, a structured AI-driven trivia game using CrewAI. It defines four agents—a Game Master, two contestants, and a Judge—each assigned specific tasks. The Game Master generates real-time trivia questions with a web search tool, the contestants respond independently, and the Judge evaluates and declares the winner. The flow function ensures data moves correctly between agents, orchestrating a fair and competitive game.

**Note:** Before running this code, install the required dependencies  
(for example, `pip install "crewai[tools]" langchain-openai openai`).  
Because open-source packages evolve quickly, Colab may show occasional
version warnings or dependency conflicts during installation. These seldom
prevent the code from running, but if something does break, try restarting
the runtime, reinstalling packages, or switching to an alternate search tool
from `crewai_tools`. If the agents appear to loop or stall, increasing
`max_iter` to 3 and enabling `verbose=True` usually helps stabilize behavior
and makes debugging easier.

In [ ]:
%%capture --no-stderr
%pip install crewai crewai-tools langchain-openai openai qdrant-client

print("----- CrewAI Installed! -----")

In [ ]:
import re
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from crewai_tools import SerperDevTool

# --- Initialize Web Search Tool ---
search_tool = SerperDevTool()

# --- Define LLMs ---
system_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7,
                        max_tokens=250)
player1_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.8,
                         max_tokens=50)
player2_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.9,
                         max_tokens=50)

# --- Define Agents ---

game_master = Agent(
    role="Game Master",
    goal="Generate trivia questions on non-political current events today.",
    backstory="An impartial host creating fair and engaging trivia games.",
    tools=[search_tool],
    max_iter = 3,
    llm=system_llm
)

player_1 = Agent(
    role="Contestant 1",
    goal="Answer trivia questions quickly using only internal knowledge.",
    backstory="A fast-thinking trivia player relying on existing knowledge.",
    max_iter = 3,
    llm=player1_llm
)

player_2 = Agent(
    role="Contestant 2",
    goal="Answer trivia questions strategically, focusing on reasoning.",
    backstory="A methodical contestant carefully formulating responses.",
    max_iter = 3,
    llm=player2_llm
)

judge = Agent(
    role="Judge",
    goal="Evaluate trivia answers and declare a winner with justification.",
    backstory="An impartial adjudicator ensuring fairness in competition.",
    max_iter = 3,
    llm=system_llm
)

# --- Define Tasks ---

generate_question_and_answer = Task(
    description="Create a trivia question on a non-political event today. "
                "Ensure the event is recent and unlikely in LLM training data. "
                "Format the response as:\n\n"
                "Question: <trivia question>\n"
                "Answer: <correct answer>",
    expected_output="A clearly formatted trivia question and correct answer.",
    agent=game_master
)

player1_answer = Task(
    description="Given the question:\n{question}\n\n"
                "Provide your best answer using only internal knowledge.",
    expected_output="A concise and accurate response to the trivia question.",
    agent=player_1
)

player2_answer = Task(
    description="Given the question:\n{question}\n\n"
                "Provide your best answer with logical reasoning.",
    expected_output="A thoughtful and well-reasoned response to the trivia "
                    "question.",
    agent=player_2
)

evaluate_and_declare_winner = Task(
    description="Given the question:\n{question}\n\n"
                "Correct answer:\n{right_answer}\n\n"
                "Player 1's answer:\n{player1_answer}\n\n"
                "Player 2's answer:\n{player2_answer}\n\n"
                "Determine the most accurate response and declare the winner.\n\n"
                "Format as:\n\n"
                "Winner: Contestant X\n"
                "Justification: <brief explanation>",
    expected_output="A clear winner declaration and reasoning.",
    agent=judge
)

# --- Define Crews ---

game_master_crew = Crew(
    agents=[game_master],
    tasks=[generate_question_and_answer],
    verbose=True
)

player1_crew = Crew(
    agents=[player_1],
    tasks=[player1_answer],
    verbose=True
)

player2_crew = Crew(
    agents=[player_2],
    tasks=[player2_answer],
    verbose=True
)

judge_crew = Crew(
    agents=[judge],
    tasks=[evaluate_and_declare_winner],
    verbose=True
)

# --- Define Flow Function to Orchestrate Contest ---

def extract_question_answer(response_text):
    """Extracts question and answer from structured text using regex."""
    match = re.search(r"Question:\s*(.*?)\s*Answer:\s*(.*)",
                      response_text, re.IGNORECASE)
    if match:
        return match.group(1).strip(), match.group(2).strip()
    return "No question generated.", "No correct answer provided."

def run_neural_duel():
    """Orchestrates the trivia contest flow between AI agents."""
    print("🎲 Starting Neural Duel!\n")

    # Step 1: Game Master generates trivia question and correct answer
    gm_results = game_master_crew.kickoff()
    print("\n🔍 Raw Game Master Results:\n", gm_results)

    # Extract question and correct answer
    question, correct_answer = extract_question_answer(str(gm_results))

    print(f"\n📝 Trivia Question: {question}")
    print(f"🤫 (Secret) Correct Answer: {correct_answer}")

    # Step 2: Players answer the question
    player1_results = player1_crew.kickoff(inputs={"question": question})
    player2_results = player2_crew.kickoff(inputs={"question": question})

    print(f"\n🏅 Contestant 1's Answer: {str(player1_results)}")
    print(f"🏅 Contestant 2's Answer: {str(player2_results)}")

    # Step 3: Judge evaluates responses and declares the winner
    judge_results = judge_crew.kickoff(inputs={
        "question": question,
        "right_answer": correct_answer,
        "player1_answer": str(player1_results),
        "player2_answer": str(player2_results)
    })

    print("\n🏆 Neural Duel Results:\n", str(judge_results))

# --- Run the Game ---
run_neural_duel()



### Listing 11-4: Generating Chapter Artwork with the Illustrator Agent
This code defines the Chapter Illustrator agent, a lightweight AI designed to create a single image for the chapter. The agent uses DALL·E as its sole tool and receives a short visual prompt describing the desired scene. When run, the agent produces a landscape-oriented, comic-style illustration of two AI robots playing chess, demonstrating how an agent can carry out a focused creative task within a larger workflow.

In [ ]:
%%capture --no-stderr
%pip install crewai crewai-tools langchain-openai openai qdrant-client

In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import DallETool

# Ensure API key is set
if "OPENAI_API_KEY" not in os.environ:
    raise SystemExit("Please set the OPENAI_API_KEY env var before running.")

# Landscape DALL·E tool
image_tool = DallETool(
    model="dall-e-3",
    size="1792x1024",   # landscape orientation
    quality="standard",
    n=1,
)

# Minimal and clear agent
illustrator_agent = Agent(
    role="Illustrator",
    goal="Generate exactly one DALL-E image when requested.",
    backstory="You convert short prompts into visual concepts.",
    tools=[image_tool],
    llm="gpt-4o-mini",
    verbose=True,
)

# Task: explicitly tell agent how to call the tool
chapter_image_task = Task(
    description=(
        "Use ONLY the Dall-E Tool.\n\n"
        "Your Action Input MUST be JSON of this form:\n"
        "{\"image_description\": \"<prompt>\"}\n\n"
        "Use this image description:\n"
        "Create a clean, comic-style image on a white background with a "
        "color palette of Primary Red, Bright Blue, Sunny Yellow, Deep Black, "
        "White, and Muted Grey. Depict two playful AI bots playing chess against "
        "each other. Landscape composition."
    ),
    expected_output="A DALL-E image URL.",
    agent=illustrator_agent,
)

chapter_image_crew = Crew(
    agents=[illustrator_agent],
    tasks=[chapter_image_task],
    process=Process.sequential,
    verbose=True,
)

def generate_chapter_image():
    result = chapter_image_crew.kickoff()
    print("\n=== Chapter Image Result ===\n")
    print(result)
    print("\n============================\n")

if __name__ == "__main__":
    generate_chapter_image()